# Teva - List
## Wichtig
* HCP & HCO manuell setzen!

In [13]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Unlock PDF
Dieser Teil entfernt das Lock des PDFs, damit das PDF in HTML exportiert werden kann

In [2]:
pdf = pikepdf.open('PKK Erfassung.Teva Pharma AG_ENG_Final_2018.pdf')
pdf.save('unlocked.pdf')

In [33]:
# Read pdf into DataFrame
columns = [58, 194, 252, 300, 384, 438, 504, 603, 642, 716, 782, 848, 933, 1006, 1094]
df = tabula.read_pdf("unlocked.pdf", pages='all', lattice=False, columns=columns, guess=False)

In [47]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'date', 'donations_grants', 'empty1', 'empty2', 'sponsorship', 'registration_fees','travel_accommodation', 'empty3', 'fees', 'related_expenses', 'total']

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export[df_export['country'] == 'Switzerland']

#Add Type
df_export = df_export.reset_index(drop=True)
index_hco = df_export[df_export['name'].str.contains("SAKK", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')
add_warning(manually=True)

#Remove Title
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Revert Name
df_export = revert_name(df_export, ', ')

#Remove empty
df_export.drop(columns=['date', 'empty0', 'empty1', 'empty2', 'empty3'], inplace=True)


#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Add Fields
df_export = add_uci(df_export)
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'teva')

Total nicht Summe der Werte
saved


In [48]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)